In [29]:
# imports
from pymongo import MongoClient
from pprint import pprint
import os
from bson.objectid import ObjectId
import pandas as pd
try:
    import ktl
    useKTL = True
except:
    print("KTL functions are not available")
    useKTL = False

KTL functions are not available


In [5]:
# establish connection with configuration database
# first you need: ssh keola@observinglogs -L 27017:localhost:27017
client = MongoClient('localhost')



In [66]:
class Instrument():
    def __init__(self):
        print("Instrument class is ready")
        
    # KTL decorator: This decorator returns the original method of the super class if KTL
    # is not defined
    @classmethod
    def ktl_decorator(cls, func):
        if useKTL:
            return func
        else:
            print("%s(%s): KTL is not available, using super method" % 
                  (cls.__name__,func.__name__))
            return getattr(cls, func.__name__)
                
    def get_outdir(self):
        """
        Return the current working directory. 
        Individual child classes can override to use outdir
        """
        return(os.getcwd())
    
    def get_all_configs(self, semester=None, progname=None):
        configurations = []
        if semester and progname:
            confs = self.db.Configurations.find({'semester': semester, 'progname': progname})
        elif semester:
            confs = self.db.Configurations.find({'semester': semester})
        elif progname:
            confs = self.db.Configurations.find({ 'progname': progname})
        else:
            confs = self.db.Configurations.find()
        configurations = list(confs)
        converted_configurations = []
        for config in configurations:
            converted_configurations.append(self.convert_configuration(config))
                        
        print(pd.DataFrame(converted_configurations))
            
        
    def get_all_projects(self, semester=None):
        projects_list = []
        if semester is None:
            projects = self.db.Configurations.find({},{'progname':1})
        else:
            projects = self.db.Configurations.find({'semester': semester}, {'progname':1})
        for project in projects:
            try:
                projects_list.append(project['progname'])
            except:
                pass
        print(set(projects_list))
        
    def get_config(self, semester=None, progname=None, statenam=None):
        config = self.db.Configurations.find(
            {'semester':semester, 
             'progname': progname,
             'statenam': statenam})
        for conf in config:
            output_config = conf
            pprint(conf)
        return(output_config)
        
    def convert_configuration(self, configuration):
        configurationDetail={}
        if not self.elements:
            print("The conversion table is not defined")
            return
        for key in self.elements.keys():
            state_keyword = self.elements[key][0]
            mongo_keyword = self.elements[key][1]
            required = self.elements[key][2]
            if required:
                try:
                    configurationDetail[state_keyword] = configuration[mongo_keyword]
                except Exception as e:
                    raise KeyError("Missing keyword %s" % (e))

            else:
                try:
                    configurationDetail[state_keyword] = configuration[mongo_keyword]
                except:
                    configurationDetail[state_keyword]=""
        configurationDetail['id']=str(configuration['_id'])
        return configurationDetail
    
    def get_state_file(self, semester=None, progname=None, statenam=None):
        config = self.db.Configurations.find(
            {'semester':semester, 
             'progname': progname,
             'statenam': statenam})
        for conf in config:
            output_config = conf
        translated_config = self.convert_configuration(conf)
        if translated_config:
            for key in translated_config:
                print("%s = %s" % (key, translated_config[key]))
    

In [67]:
class LRIS(Instrument):
    def __init__(self):
        self.db = client.LRIS
        self.set_conversion_elements()
        super(LRIS, self).__init__()
    
    @Instrument.ktl_decorator
    def get_outdir(self):
        lris = ktl.cache('lris')
        outdir = lris['outdir'].read()
        return(outdir)
    
    def set_conversion_elements(self):
        self.elements = {
            '1': ['statenam','statenam',False],
            '2': ['GRISNAME','bluegrism',False],
            '3': ['DICHNAME','dichroic',False],
            '4': ['BLUFILT','filter',False],
            '5': ['GRANAME','redgrism',False],
            '6': ['SLITNAME','slitmask',False],
            }
    

class MOSFIRE(Instrument):
    def __init__(self):
        self.db = client.MOSFIRE
        super(MOSFIRE, self).__init__()
        
    @Instrument.ktl_decorator
    def get_outdir(self):
        mosfire = ktl.cache('mosfire')
        outdir = mosfire['outdir'].read()
        return(outdir)
    
class KCWI(Instrument):
    def __init__(self):
        self.db = client.KCWI
        self.set_conversion_elements()
        super(KCWI, self).__init__()
        
    def set_conversion_elements(self):
        self.elements = {
            '1': ['statenam','statenam',True],
            '2': ['image_slicer','image_slicer',False],
            '3': ['filterb','filterb',False],
            '4': ['gratingb','gratingb',False],
            '5': ['nsmaskb','nsmaskb',False],
            '6': ['ampmodeb','ampmodeb',False],
            '7': ['gainmulb','gainmulb',False],
            '8': ['ccdmodeb','ccdmodeb',False],
            '9': ['binningb','binningb',False],
            '10':['cal_mirror','cal_mirror',False],
            '11':['polarizer','polarizer',False],
            '12':['cwaveb','cwaveb',False],
            '13':['pwaveb','pwaveb',False],
            '14':['progname','progname',False],
            '15':['camangleb','camangleb',False],
            '16':['focusb','focusb',False]
            }
        

    @Instrument.ktl_decorator
    def get_outdir(self):
        kbds = ktl.cache('lbds')
        outdir = kbds['outdir'].read()
        return(outdir)

Instrument(get_outdir): KTL is not available, using super method
Instrument(get_outdir): KTL is not available, using super method
Instrument(get_outdir): KTL is not available, using super method


In [68]:
lris = LRIS()

Instrument class is ready


In [69]:
lris.get_outdir()

'/Users/lrizzi/Python_Projects/ConfigurationClient'

In [70]:
lris.get_all_projects(semester='2018B')

{'C999', 'K999', '_test'}


In [71]:
lris.get_all_configs()

       BLUFILT DICHNAME    GRANAME   GRISNAME     SLITNAME  \
0                                                            
1                                                            
2                                                            
3                                                            
4                                                            
5                                                            
6                                                            
7                                                            
8   NB6741_Red     D560  1200/7500  1200/3400     long_8.7   
9        R_Red     D500   600/7500   300/5000     long_1.5   
10                                                           
11                                                           
12      g_Blue     D560   400/8500   600/4000     long_1.0   
13                 D460              400/3400     long_1.0   
14                 D560              600/4000     long_1.0   
15      

In [11]:
lris.get_config('2018B', 'C999', 'CheckID')

{'_id': ObjectId('5bca52c54eae3539aa63171d'),
 'bluegrism': '600/4000',
 'detector': 'Blue',
 'dichroic': 'D560',
 'filter': 'g_Blue',
 'polarimeter': 'Off',
 'progname': 'C999',
 'redgrism': '400/8500',
 'semester': '2018B',
 'slitmask': 'long_1.0',
 'statenam': 'CheckID'}


{'_id': ObjectId('5bca52c54eae3539aa63171d'),
 'statenam': 'CheckID',
 'detector': 'Blue',
 'dichroic': 'D560',
 'filter': 'g_Blue',
 'bluegrism': '600/4000',
 'redgrism': '400/8500',
 'slitmask': 'long_1.0',
 'polarimeter': 'Off',
 'progname': 'C999',
 'semester': '2018B'}

In [10]:
mosfire = MOSFIRE()

Instrument class is ready


In [11]:
mosfire.get_all_projects(semester='2018B')

{'K999'}


In [12]:
mosfire.get_all_configs('K999')

In [13]:
mosfire.get_config('2018B','K999','Josh Test 1')

{'_id': ObjectId('5bce74b04eae3539aa631721'),
 'coadds': '1',
 'dither': "ABA'B'",
 'mdcname': 'qwer',
 'obsmode': 'J2-spectroscopy',
 'progname': 'K999',
 'semester': '2018B',
 'statenam': 'Josh Test 1'}


{'_id': ObjectId('5bce74b04eae3539aa631721'),
 'statenam': 'Josh Test 1',
 'obsmode': 'J2-spectroscopy',
 'mdcname': 'qwer',
 'coadds': '1',
 'progname': 'K999',
 'semester': '2018B',
 'dither': "ABA'B'"}

In [15]:
kcwi = KCWI()

Instrument class is ready


In [16]:
kcwi.get_all_configs(progname='C343')

                        _id  ampmodeb binningb cal_mirror camangleb  ccdmodeb  \
0  5a861aeace9e24352e97b746         9      2,2        Sky                   0   
1  5a861b88ce9e24352e97b748         9      2,2        Sky                   0   
2  5a861bc5ce9e24352e97b749         9      2,2        Sky                   0   
3  5a861c76ce9e24352e97b74b         9      2,2        Sky                   0   
4  5a861cc3ce9e24352e97b74c         9      2,2        Sky                   0   
5  5a861d99ce9e24352e97b74d         9      2,2        Sky                   0   
6  5a88027bce9e242955e92581         9      2,2        Sky                   0   
7  5a880283ce9e242955e92582         9      2,2        Sky                   0   
8  5a88e835ce9e2472c8aa87a2         9      2,2        Sky                   0   

  cwaveb filterb focusb  gainmulb gratingb image_slicer nsmaskb polarizer  \
0   4960   KBlue               10      BH3       Medium    Open       Sky   
1   5100   KBlue               10  

In [16]:
conf = kcwi.get_config(progname='C343',statenam='NGC628_M_BH3_4960')

{'_id': ObjectId('5a861aeace9e24352e97b746'),
 'ampmodeb': 9,
 'binningb': '2,2',
 'cal_mirror': 'Sky',
 'camangleb': '',
 'ccdmodeb': 0,
 'cwaveb': '4960',
 'filterb': 'KBlue',
 'focusb': '',
 'gainmulb': 10,
 'gratingb': 'BH3',
 'image_slicer': 'Medium',
 'nsmaskb': 'Open',
 'polarizer': 'Sky',
 'progname': 'C343',
 'pwaveb': '4960',
 'statenam': 'NGC628_M_BH3_4960'}


In [32]:
kcwi.get_state_file(progname='C343', statenam='NGC628_M_BH3_4960')

statenam = NGC628_M_BH3_4960
image_slicer = Medium
filterb = KBlue
gratingb = BH3
nsmaskb = Open
ampmodeb = 9
gainmulb = 10
ccdmodeb = 0
binningb = 2,2
cal_mirror = Sky
polarizer = Sky
cwaveb = 4960
pwaveb = 4960
progname = C343
camangleb = 
focusb = 
id = 5a861aeace9e24352e97b746


In [29]:
lris.get_state_file('2018B', 'C999', 'CheckID')

statenam = CheckID
GRISNAME = 600/4000
DICHNAME = D560
BLUFILT = g_Blue
GRANAME = 400/8500
SLITNAME = long_1.0
id = 5bca52c54eae3539aa63171d


In [24]:
lris.set_conversion_elements()

In [25]:
lris.elements

{'1': ['statenam', 'statenam', True],
 '2': ['GRISNAME', 'bluegrism', False],
 '3': ['DICHNAME', 'dichroic', False],
 '4': ['BLUFILT', 'filter', False],
 '5': ['GRANAME', 'redgrism', False],
 '6': ['SLITNAME', 'slitmask', False]}